In [28]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import praw
from datetime import datetime, timedelta
import yfinance as yf

In [2]:
tokenizer = BertTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = BertForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

In [19]:
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment = torch.argmax(probs).item()
    return ['Negative', 'Neutral', 'Positive'][sentiment]

In [20]:
def fetch_news(stock_name):
    print("\n🔍 Fetching News Articles...")
    url = f"https://news.google.com/search?q={stock_name}%20stock&hl=en-IN&gl=IN&ceid=IN:en"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    links = soup.select("a.DY5T1d")
    
    news_texts = []
    for link in links[:3]:  # Limit to top 3
        article_url = "https://news.google.com" + link["href"][1:]
        article = Article(article_url)
        try:
            article.download()
            article.parse()
            news_texts.append(article.text)
        except:
            continue
    return news_texts

In [21]:
def fetch_reddit(stock_name):
    print("🔍 Fetching Reddit Posts...")
    reddit = praw.Reddit(client_id="Your_client_id",
                         client_secret="your_secret_id",
                         user_agent="stock-sentiment-script")
    posts = []
    for post in reddit.subreddit("stocks").search(stock_name, limit=3):
        posts.append(post.title + " " + post.selftext)
    return posts

In [29]:
def summarize_and_analyze(texts):
    combined_text = " ".join(texts)
    summary = combined_text[:500] + "..."  # Simple summary
    sentiment = analyze_sentiment(summary)
    return summary, sentiment

In [30]:
def is_valid_stock(stock_name):
    try:
        stock = yf.Ticker(stock_name)
        hist = stock.history(period="1d")
        return not hist.empty
    except:
        return False

In [31]:
def main():
    stock_name = input("Enter the stock name or symbol: ").upper()

    if not is_valid_stock(stock_name):
        print("❌ It is not a valid stock name.")
        return

    news = fetch_news(stock_name)
    reddit = fetch_reddit(stock_name)

    all_texts = news + reddit 
    if not all_texts:
        print("❌ No data found for the given stock.")
        return

    print("\n📄 Summarized Sentiment News:")
    summary, sentiment = summarize_and_analyze(all_texts)
    print(summary)

    print("\n📊 Sentiment Analysis Result:")
    if sentiment == 'Positive':
        print("✅ The stock price might go UP.")
    elif sentiment == 'Negative':
        print("⚠️ The stock price might go DOWN.")
    else:
        print("🤔 Can't say anything now, check the technical analysis.")

if __name__ == "__main__":
    main()

Enter the stock name or symbol:  aapl



🔍 Fetching News Articles...
🔍 Fetching Reddit Posts...

📄 Summarized Sentiment News:
Should I cash out AAPL or hold? As the title says -- -I have about 1200 shares, purchased about 15 years ago. Waffling on whether to cash out now before the 50% additional tariffs become a reality or hold until some future date when things return to "normal". Are you worried about AAPL long term?  Now I’m not saying to sell all of your Apple stock immediately. However, given that:

1. The Vision Pro was a dud, they’re pulling back manufacturing/production

2. iPhone sales are down almost 10%

3....

📊 Sentiment Analysis Result:
✅ The stock price might go UP.
